# Final Project: Deeplearning to detect eye diseses

First using a pretrained CNN model, retraining with own pictures

# Load Packages

In [18]:
import pandas as pd
import numpy as np

# deep learning
import keras
from keras.models import Model, Sequential
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
#from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
# ResNet50 model
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
import keras_tuner as kt


# data visualization stack
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

# evaluating
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# others
import os

# Load Data

In [ ]:
# connect to drive to access data in colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# get classes from train image set in colab
# CLASSES = os.listdir('/content/drive/My Drive/Colab Notebooks/data_final_project/train')
# CLASSES

In [7]:
# get classes from train image set locally
CLASSES = os.listdir('data/train')
CLASSES

['cataract', 'glaucoma', 'diabetes', 'normal']

In [8]:
# instantiate image data generator
data_gen = image.ImageDataGenerator(
    # preprocessing function for all images
    preprocessing_function=preprocess_input
)

### Load Train Data

In [9]:
# load adn preprocess train data
train_data_gen = data_gen.flow_from_directory(
        directory='data/train',
        class_mode="categorical",
        classes=CLASSES,
        batch_size=400,
        target_size=(224,224)
)

Found 400 images belonging to 4 classes.


In [10]:
# crate xtrain and y train
xtrain, ytrain = next(train_data_gen)
xtrain.shape, ytrain.shape

((400, 224, 224, 3), (400, 4))

## Load Test Data

In [11]:
# preprocess test data 
test_data_gen = data_gen.flow_from_directory(
        directory='data/test',
        class_mode="categorical",
        classes=CLASSES,
        batch_size=40,
        target_size=(224,224)
)

Found 40 images belonging to 4 classes.


In [12]:
# create test set
xtest, ytest = next(test_data_gen)
xtest.shape, ytest.shape

((40, 224, 224, 3), (40, 4))

# Model Building

In [14]:
resnet50_model = ResNet50V2(
    include_top=True,
    weights="imagenet",
    classes=1000
)

102869336/102869336 [==============================] - 15s 0us/step


In [17]:
resnet50_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

                                                                                                  
 conv2_block3_preact_relu (Acti  (None, 56, 56, 256)  0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 56, 56, 64)   16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_blo

 conv3_block2_2_conv (Conv2D)   (None, 28, 28, 128)  147456      ['conv3_block2_2_pad[0][0]']     
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 28, 28, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block1_1_conv (Conv2D)   (None, 14, 14, 256)  131072      ['conv4_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block1_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block1_2_pad (ZeroPaddin  (None, 16, 16, 256)  0          ['conv4_block1_1_relu[0][0]']    
 g2D)     

                                                                                                  
 conv4_block3_2_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block3_3_conv (Conv2D)   (None, 14, 14, 1024  263168      ['conv4_block3_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv4_block3_out (Add)         (None, 14, 14, 1024  0           ['conv4_block2_out[0][0]',       
                                )                                 'conv4_block3_3_conv[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block6_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block6_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block6_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block6_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block6_2_pad (ZeroPaddin  (None, 16, 16, 256)  0          ['conv4_block6_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv4_block6_2_conv (Conv2D)   (None, 7, 7, 256)    589824      ['conv4_block6_2_pad[0][0]']     
          

                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 7, 7, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_out (Add)         (None, 7, 7, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_conv[0][0]']    
                                                                                                  
 conv5_block3_preact_bn (BatchN  (None, 7, 7, 2048)  8192        ['conv5_block2_out[0][0]']       
 ormalization)                                                                                    
                                                                                                  
 conv5_block3_preact_relu (Acti  (None, 7, 7, 2048)  0           ['conv5_block3_preact_bn[0][0]'] 
 vation)  

In [13]:
# create base model
base_model = ResNet50V2(
    weights='imagenet', 
    include_top=False,  # removal of final dense layers
    pooling='avg',      # average pooling to last convolutional layer's ouput
    input_shape=(224,224,3) # ignored if input tensor is provided
)

2023-03-04 15:20:32.917192: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


94668760/94668760 [==============================] - 13s 0us/step


In [15]:
# freeze base model
base_model.trainable = False

In [16]:
# check base model
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

                                                                                                  
 conv2_block3_preact_relu (Acti  (None, 56, 56, 256)  0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 56, 56, 64)   16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_blo

 conv3_block2_2_conv (Conv2D)   (None, 28, 28, 128)  147456      ['conv3_block2_2_pad[0][0]']     
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 28, 28, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block1_1_conv (Conv2D)   (None, 14, 14, 256)  131072      ['conv4_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block1_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block1_2_pad (ZeroPaddin  (None, 16, 16, 256)  0          ['conv4_block1_1_relu[0][0]']    
 g2D)     

                                                                                                  
 conv4_block3_2_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block3_3_conv (Conv2D)   (None, 14, 14, 1024  263168      ['conv4_block3_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv4_block3_out (Add)         (None, 14, 14, 1024  0           ['conv4_block2_out[0][0]',       
                                )                                 'conv4_block3_3_conv[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block6_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block6_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block6_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block6_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block6_2_pad (ZeroPaddin  (None, 16, 16, 256)  0          ['conv4_block6_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv4_block6_2_conv (Conv2D)   (None, 7, 7, 256)    589824      ['conv4_block6_2_pad[0][0]']     
          

                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 7, 7, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_out (Add)         (None, 7, 7, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_conv[0][0]']    
                                                                                                  
 conv5_block3_preact_bn (BatchN  (None, 7, 7, 2048)  8192        ['conv5_block2_out[0][0]']       
 ormalization)                                                                                    
                                                                                                  
 conv5_block3_preact_relu (Acti  (None, 7, 7, 2048)  0           ['conv5_block3_preact_bn[0][0]'] 
 vation)  

In [19]:
class MyHyperModel(kt.HyperModel):
    '''
    creates model to optimize hyperparameter
    '''
    def build(self, hp):
        # clear seesion
        K.clear_session()
        # create sequential model
        model = keras.Sequential()
        # add base model
        model.add(base_model)
        
        # add dense layer with hp opit for units
        # out from pretrained model 2048/4 = 512
        model.add(keras.layers.Dense(
                units=hp.Choice("units",[512,256,128,64]),
                activation="relu",))
        
        # add dropout 
        model.add(keras.layers.Dropout(rate=0.5))
        
        # add output layer 
        model.add(keras.layers.Dense(4, activation="softmax"))
            
            
        # compile model 
        model.compile(    
          optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), 
          loss=keras.losses.categorical_crossentropy,
          metrics=[keras.metrics.categorical_accuracy]
        )
        
        return model

    def fit(self, hp, model, *args, **kwargs):
        '''
        function to optimize batch size
        '''
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [10,20]),
            **kwargs,
        )

In [20]:
# early stopping

stop_early = keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10
)

## Hyperparameter Tuning - Random Search

In [21]:
# instantiate RandomSearch Tuner

tuner_RS = kt.RandomSearch(
    MyHyperModel(),
    objective='val_categorical_accuracy',
    max_trials=10,
    overwrite=True,
    directory='test_3',
    project_name='tune_hypermodel',
)

In [22]:
# hypertuning settings summary
tuner_RS.search_space_summary()

Search space summary
Default search space size: 2
units (Choice)
{'default': 512, 'conditions': [], 'values': [512, 256, 128, 64], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [23]:
tuner_RS.search(
    xtrain, 
    ytrain,
    epochs=25,
    validation_split=0.2,
    callbacks=[stop_early]
)

Trial 10 Complete [00h 08m 03s]
val_categorical_accuracy: 0.637499988079071

Best val_categorical_accuracy So Far: 0.675000011920929
Total elapsed time: 01h 32m 20s
INFO:tensorflow:Oracle triggered exit


In [24]:
best_hp_rs = tuner_RS.get_best_hyperparameters(1)[0]

In [25]:
best_hp_rs.get('units')

512

In [26]:
best_hp_rs.get('learning_rate')

0.01

In [27]:
best_hp_rs.get('batch_size')

10

In [28]:
rs_model = tuner_RS.hypermodel.build(best_hp_rs)
rs_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 24,615,940
Trainable params: 1,051,140
Non-trainable params: 23,564,800
_________________________________________________________________


In [29]:
rs_model.fit(
    xtrain,
    ytrain,
    epochs=100,
    batch_size=10,
    callbacks=[stop_early],
    validation_split=0.2
)

Epoch 1/100
32/32 [==============================] - 25s 707ms/step - loss: 10.4080 - categorical_accuracy: 0.4156 - val_loss: 7.2294 - val_categorical_accuracy: 0.5000
Epoch 2/100
32/32 [==============================] - 23s 722ms/step - loss: 3.1376 - categorical_accuracy: 0.5750 - val_loss: 1.1224 - val_categorical_accuracy: 0.5375
Epoch 3/100
32/32 [==============================] - 22s 692ms/step - loss: 0.9066 - categorical_accuracy: 0.6062 - val_loss: 0.8865 - val_categorical_accuracy: 0.6500
Epoch 4/100
32/32 [==============================] - 22s 686ms/step - loss: 0.8099 - categorical_accuracy: 0.6438 - val_loss: 1.0110 - val_categorical_accuracy: 0.5375
Epoch 5/100
32/32 [==============================] - 23s 723ms/step - loss: 0.8475 - categorical_accuracy: 0.6469 - val_loss: 0.9001 - val_categorical_accuracy: 0.5125
Epoch 6/100
32/32 [==============================] - 24s 764ms/step - loss: 0.7970 - categorical_accuracy: 0.6625 - val_loss: 0.8468 - val_categorical_accuracy

## Evaluate Model - Random Search

In [ ]:
# show loss history
plt.plot(rs_model.history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
ypred = np.round(rs_model.predict(xtest), 3)

In [ ]:
df_pred = pd.DataFrame(
    columns=CLASSES,
    data=ypred
)

In [ ]:
# convert vectorized classes into strings
df_pred['class'] = \
[''.join([i*j for (i,j) in zip(CLASSES,vector)]) for vector in ytest.astype(int)]

df_pred

In [ ]:
rs_model.evaluate(xtrain, ytrain)

In [ ]:
rs_model.evaluate(xtest, ytest)

In [ ]:
# save model
rs_model.save('models/model_ResNet_local_rs.h5')

## Hyperparameter Tuning - Grid Search

In [ ]:
# von Sebastian
tuner_gs = kt.GridSearch(
    hypermodel=HyperModel,
    objective='val_categorical_accuracy',
    project_name='./drive/MyDrive/Colab Notebooks/data/kt_files'
)

In [ ]:
best_hp_gs = tuner_RS.get_best_hyperparameters(1)[0]

In [ ]:
best_hp_gs.get('units')

In [ ]:
best_hp_gs.get('learning_rate')

In [ ]:
best_hp_gs.get('batch_size')

In [ ]:
gs_model = tuner_gs.hypermodel.build(best_hp)
gs_model.summary()

In [ ]:
gs_model.fit(
    xtrain,
    ytrain,
    epochs=100,
    batch_size=10,
    callbacks=[stop_early],
    validation_split=0.2
)

## Evaluate Model - Grid Search

In [ ]:
# show loss history
plt.plot(gs_model.history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
ypred_gs = np.round(gs_model.predict(xtest), 3)

In [ ]:
gs_model.evaluate(xtrain, ytrain)

In [ ]:
gs_model.evaluate(xtest, ytest)

In [ ]:
# save model
gs_model.save('models/model_ResNet_local_gs.h5')

## Confusion Matrix for best Model

## Test Images

# Save Model